To be split from `1.2` which includes both preprocessing AND QuestionAnswering modeling

# Training a single line!! 

In [ ]:
from transformers import AutoModelForQuestionAnswering, Trainer, TrainingArguments, default_data_collator

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")
data_collator = default_data_collator

In [ ]:
single_line_train_dataset = tokenized_train_dataset.select(range(32))

In [ ]:
# define training args
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    weight_decay=0.01,
)

# create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=single_line_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.__dict__.keys()

In [ ]:
trainer.train()

# Actual Training

In [ ]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters = {
    'epochs': 3,
     'train_batch_size': 32,
     'model_name':'distilbert-base-uncased'
}
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            instance_type='ml.g4dn.xlarge',
                            instance_count=1,
                            role=role,
                            transformers_version='4.12',
                            pytorch_version='1.9',
                            py_version='py38',
                            hyperparameters = hyperparameters)

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

# Evaluate

In [ ]:
resource_tags = [
    {"Key":'Project', "Value": 'praekelt-skoll'}, 
    {"Key":'BillingCode', "Value":'praekelt-skoll'}
]
predictor = huggingface_estimator.deploy(1, instance_type='ml.m5.xlarge', tags=resource_tags)

In [ ]:
test_dataset_renamed = test_dataset.rename_column('faq_content_to_send', 'context').remove_columns(['label', 'faq_title', 'faq_id', '__index_level_0__'])

In [ ]:
test_dataset_renamed

In [ ]:
predictions

In [ ]:
pred_results = {
    'faq_id': [],
    'actual': [],
    'predicted': [],
    'question': [],
    'context': [],
}
for i, example in enumerate(test_dataset_renamed):
    prediction = predictor.predict(example)
    pred_results['faq_id'].append(test_dataset[i]['faq_id'])
    pred_results['actual'].append(test_dataset[i]['label'])
    pred_results['predicted'].append(prediction['score'])
    pred_results['question'].append(example['question'])
    pred_results['context'].append(test_dataset[i]['faq_content_to_send'])

In [ ]:
pred = pd.DataFrame(pred_results)
pred.plot.scatter(x='actual', y='predicted')

In [ ]:
i = 30
predictions = predictor.predict(test_dataset_renamed[i])

print(f"Question: {test_dataset[i]['question']}")
print(f"Label: {test_dataset[i]['label']:.0f} / Predicted: {predictions['score']:.2f}")
context = test_dataset[i]['faq_content_to_send']
print(f"Answer: {context[predictions['start']: predictions['end']]}")
print(f"Context: {context}")

In [ ]:
df_merged[df_merged.faq_id == 77]

In [ ]:
all_samples[all_samples.faq_id == 77]